<a href="https://colab.research.google.com/github/shivam-sunita-puri/TIYA/blob/main/Calculate_average_heart_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
########################################################
# Calculate average heart rate for the period specified
########################################################

# Substitute YYY below with token received
access_token='YYY'

# Change date below to correct date
url = "https://api.fitbit.com/1/user/-/activities/heart/date/2024-11-05/1d/1sec.json"  # Replace date as needed

# Call the function below with start_time and end_time given as strings in HH:MM:SS format (24 hours notation)

# If all the data in the time window is already on the FitBit website, the function returns the avergae heart rate in the time window

# If some of the data still weren't uploaded to FitBit website (data is uploaded every 15-20 minutes), there will be an error message and the function will be returning -99

# import packages
import pprint
import requests
import time
from datetime import datetime, timedelta
import pandas as pd

# set correct timezone
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Australia/Melbourne /etc/localtime
!date

def calculate_average_heart_rate(start_time, end_time):
  headers = {'Authorization': f'Bearer {access_token}'}

  response = requests.get(url, headers=headers)
  data = response.json()

  if response.status_code == 200:
    #  pprint.pprint(data['activities-heart-intraday']['dataset'][-1:])
    df = pd.DataFrame(data['activities-heart-intraday']['dataset'])

    # Parse the dates into datetime objects
    time_format = "%H:%M:%S"  # Specify the format matching YY:MM:DD
    df['parsed_time']=[datetime.strptime(time_string,time_format) for time_string in df['time']]

    parsed_start_time = datetime.strptime(start_time, time_format)
    parsed_end_time = datetime.strptime(end_time, time_format)

    # Compare the dates
    if parsed_start_time < df['parsed_time'][0]:
        print(f"start time {parsed_start_time} is earlier than the time stamp of the first data sample {df['parsed_time'].iloc[0]}")
        return -99
    elif parsed_end_time > df['parsed_time'].iloc[-1]:
        print(f"end time {parsed_end_time} is later than the time stamp of the last data sample {df['parsed_time'].iloc[-1]}")
        return -99


    times = [row['time'] for index, row in df.iterrows() if row['parsed_time'] >= parsed_start_time and row['parsed_time'] <= parsed_end_time]
    print(f"DEBUG INFORMATION: {len(times)} time stamps averaged:")
    print(f"DEBUG INFORMATION: {times}")

    values_to_average = [row['value'] for index, row in df.iterrows() if row['parsed_time'] >= parsed_start_time and row['parsed_time'] <= parsed_end_time]
    print(f"DEBUG INFORMATION: {values_to_average}")

    return sum(values_to_average) / len(values_to_average)

  else:
    pprint.pprint(data)

def last_X_minutes_available(minutes):
  headers = {'Authorization': f'Bearer {access_token}'}

  response = requests.get(url, headers=headers)
  data = response.json()

  if response.status_code == 200:
    #  pprint.pprint(data['activities-heart-intraday']['dataset'][-1:])
    df = pd.DataFrame(data['activities-heart-intraday']['dataset'])

    # Parse the dates into datetime objectsd
    time_format = "%H:%M:%S"  # Specify the format matching YY:MM:DD
    df['parsed_time']=[datetime.strptime(time_string,time_format) for time_string in df['time']]

    last_sample_time = df['time'].iloc[-1]

    X_minutes_before_parsed_time = df['parsed_time'].iloc[-1] - timedelta(minutes=minutes)

    first_sample_parsed_time = next((time_parsed for time_parsed in df['parsed_time'] if time_parsed >= X_minutes_before_parsed_time), None)

    first_sample_time = first_sample_parsed_time.strftime("%H:%M:%S")

    return first_sample_time, last_sample_time

  else:
    pprint.pprint(data)

Tue Nov  5 12:29:17 PM AEDT 2024


In [25]:
# Example usage - calculate average in certain time range

rate = calculate_average_heart_rate('09:50:00','09:51:00')

print(rate)

13 time stamps averaged:
['09:50:00', '09:50:05', '09:50:10', '09:50:15', '09:50:20', '09:50:25', '09:50:30', '09:50:35', '09:50:40', '09:50:45', '09:50:50', '09:50:55', '09:51:00']
[125, 125, 126, 124, 122, 122, 118, 118, 121, 123, 120, 115, 110]
120.6923076923077


In [43]:
# Example usage - every minute, calculate the average in the last 5 minutes of available data

minutes = 5

while True:
  first_sample_time, last_sample_time = last_X_minutes_available(5)
  print(f"Last available time stamp: {last_sample_time}")
  print(f"Time stamp {minutes} minutes beforehand: {first_sample_time}")
  rate = calculate_average_heart_rate(first_sample_time,last_sample_time)
  print(f"Average heart rate in this period: {rate}")
  print("")
  time.sleep(60)

Last available time stamp: 12:23:33
Time stamp 5 minutes beforehand: 12:18:33
DEBUG INFORMATION: 37 time stamps averaged:
DEBUG INFORMATION: ['12:18:33', '12:18:38', '12:18:43', '12:18:53', '12:19:03', '12:19:13', '12:19:18', '12:19:23', '12:19:33', '12:19:38', '12:19:43', '12:19:58', '12:20:08', '12:20:13', '12:20:18', '12:20:23', '12:20:28', '12:20:43', '12:20:48', '12:20:58', '12:21:03', '12:21:08', '12:21:23', '12:21:38', '12:21:43', '12:21:48', '12:21:58', '12:22:08', '12:22:13', '12:22:23', '12:22:33', '12:22:43', '12:22:58', '12:23:13', '12:23:23', '12:23:28', '12:23:33']
DEBUG INFORMATION: [86, 87, 88, 89, 88, 87, 87, 84, 85, 84, 83, 84, 82, 83, 82, 81, 82, 83, 84, 87, 89, 89, 87, 88, 90, 91, 89, 88, 89, 87, 88, 89, 90, 88, 89, 90, 90]
Average heart rate in this period: 86.67567567567568

Last available time stamp: 12:23:33
Time stamp 5 minutes beforehand: 12:18:33
DEBUG INFORMATION: 37 time stamps averaged:
DEBUG INFORMATION: ['12:18:33', '12:18:38', '12:18:43', '12:18:53', '1

KeyboardInterrupt: 

In [33]:
first_sample_time

'12:06:04'

In [34]:
last_sample_time

'12:09:04'

In [35]:
rate = calculate_average_heart_rate(*last_X_minutes_available(3))

print(rate)

23 time stamps averaged:
['12:06:04', '12:06:14', '12:06:19', '12:06:24', '12:06:29', '12:06:34', '12:06:44', '12:06:49', '12:06:54', '12:06:59', '12:07:04', '12:07:19', '12:07:24', '12:07:29', '12:07:44', '12:07:49', '12:08:04', '12:08:19', '12:08:24', '12:08:39', '12:08:54', '12:08:59', '12:09:04']
[73, 71, 72, 71, 72, 71, 74, 75, 73, 71, 70, 70, 72, 77, 75, 71, 71, 71, 72, 72, 71, 70, 69]
71.91304347826087
